In [1]:
import pandas as pd
import re
import plotly.express as px

# 读取 origin 数据
file_path = 'e2e_coco_lora(old).csv'
df = pd.read_csv(file_path)

# 提取并转换数据
df['Tensor_Value'] = df.iloc[:, 4].apply(lambda x: float(re.findall(r"[-+]?\d*\.\d+|\d+", x)[0]))
df_filtered = df[['Tensor_Value', df.columns[2], df.columns[5], df.columns[0]]].copy()
df_filtered.columns = ['Tensor', 'Q', 'Accuracy', 'Type']

# 根据类型动态分配颜色
df_filtered['Type'] = df_filtered['Type'].apply(lambda x: 'Dynamic-old' if x == 'dynamic' else 'E2E-old')


# 读取 Baseline 数据
baseline_path = 'test_coco_zeroshot_lora.csv'
df_baseline = pd.read_csv(baseline_path)
df_baseline_filtered = df_baseline.iloc[0::2, [0, 1]].copy()

# 创建Baseline DataFrame
df_baseline_filtered.columns = ['Tensor', 'Accuracy']
df_baseline_filtered['Q'] = 0.67  # 假设 Baseline 的 Q 值
df_baseline_filtered['Type'] = 'Baseline'

# 合并数据
df_final = pd.concat([df_filtered, df_baseline_filtered], ignore_index=True)


# 读取 LoRa 数据
file_path = 'e2e_coco_lora.csv'
df = pd.read_csv(file_path)

# 提取并转换数据
df['Tensor_Value'] = df.iloc[:, 4].apply(lambda x: float(re.findall(r"[-+]?\d*\.\d+|\d+", x)[0]))
df_lora_filtered = df[['Tensor_Value', df.columns[2], df.columns[5], df.columns[0]]].copy()
df_lora_filtered.columns = ['Tensor', 'Q', 'Accuracy', 'Type']

# 根据类型动态分配颜色
df_lora_filtered['Type'] = df_lora_filtered['Type'].apply(lambda x: 'Dynamic' if x == 'dynamic' else 'E2E')

# 合并数据
df_final = pd.concat([df_final, df_lora_filtered], ignore_index=True)

# 创建3D散点图
fig = px.scatter_3d(df_final, x='Tensor', y='Q', z='Accuracy', color='Type',
                    labels={'Tensor': 'Avg. Layers', 'Q': 'Query cost', 'Accuracy': 'R@1'},
                    title='3D Scatter Plot of E2E exps',
                    color_discrete_map={'Dynamic': 'red', 'E2E': 'blue', 'Baseline': 'black', 'Dynamic-old': 'green', 'E2E-old': 'yellow'})
# 设置数据点大小
fig.update_traces(marker=dict(size=2))  # 这里的数字可以根据您的需求调整，数值越小点越小
# 显示图表
fig.show()